In [1]:
import TensorFlow

In [3]:
let hiddenSize: Int = 10

struct Model: Layer {
  var layer1 = Dense<Float>(inputSize: 4, outputSize: hiddenSize, activation: relu)
  var layer2 = Dense<Float>(inputSize: hiddenSize,outputSize: hiddenSize, activation:relu)
  var layer3 = Dense<Float>(inputSize: hiddenSize, outputSize: 3, activation: identity)

  @differentiable
  func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
    return input.sequenced(through: layer1, layer2, layer3)
  }
}

In [7]:
var classifier = Model()
let optimizer = SGD(for: classifier, learningRate: 0.02)
Context.local.learningPhase = .training

let x: Tensor<Float> = Tensor(randomNormal: [100,4])
let y: Tensor<Int32> = Tensor(randomUniform: [100])

In [8]:
x

[[  -0.2366522,   0.38420665,   -1.2170726,  0.031472195],
 [   2.2577715,  -0.36172587, 0.0025076442,    1.3670573],
 [   0.5575517,   0.20541692,    1.6736583, -0.096976005],
 [   1.9543016,  -0.13213885,    0.5769991,   -0.4048143],
 [   1.3719854,     -2.92485,   -1.0687624,  0.068052374],
 [   -0.431636,   -1.8826449,   -0.6071108,  -0.28001437],
 [  0.06404338,   -0.5069285, -0.007832612,  -0.18222228],
 [  -1.3797941,   -0.5704593,   -1.9102843,  0.029391441],
 [   1.4884613,   -1.0407106,  -0.49925455,    0.6469843],
 [   1.2615916,    1.4858006, -0.051375736,      -0.9544],
 [   0.5401154, -0.016240878,   0.34931973,    1.3026577],
 [   0.5011489,    1.2021132,    0.6600342,   0.22635809],
 [ -0.40629143,    0.3381891,   -1.1181967,   0.90427834],
 [ -0.06036658,  -0.37016264,  -0.61451817,  -0.12572794],
 [  0.00598374,    0.7680666,   -1.7290308,  -0.16675787],
 [   1.2518748,   -0.3830106,    -1.294969,    0.8761465],
 [   -1.109355,    1.0432155,  -0.05817367,   -0.1361037

In [9]:
y

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
for _ in 0..<1000 {
  let grad_model = gradient(at:classifier) { classifier -> Tensor<Float> in
      let y_pred = classifier(x)
      let loss = softmaxCrossEntropy(logits: y_pred , labels:y )
      print("Loss: \(loss)")
      return loss
   }
   optimizer.update(&classifier, along:grad_model )
}

Loss: 2.0101874
Loss: 1.8923478
Loss: 1.785499
Loss: 1.6878712
Loss: 1.5990916
Loss: 1.5181682
Loss: 1.4439408
Loss: 1.3756387
Loss: 1.312568
Loss: 1.2541703
Loss: 1.1997676
Loss: 1.1497087
Loss: 1.1030143
Loss: 1.0594975
Loss: 1.0189471
Loss: 0.9811058
Loss: 0.9454544
Loss: 0.9115131
Loss: 0.8794412
Loss: 0.8488296
Loss: 0.8196754
Loss: 0.79203886
Loss: 0.7656896
Loss: 0.7406723
Loss: 0.71682566
Loss: 0.6939703
Loss: 0.6716098
Loss: 0.64995956
Loss: 0.62920684
Loss: 0.6094829
Loss: 0.590753
Loss: 0.5728599
Loss: 0.555593
Loss: 0.538988
Loss: 0.5231141
Loss: 0.50792605
Loss: 0.49341947
Loss: 0.47954357
Loss: 0.46626103
Loss: 0.45350897
Loss: 0.44127545
Loss: 0.42950127
Loss: 0.41821995
Loss: 0.4073726
Loss: 0.39693582
Loss: 0.38689643
Loss: 0.3772317
Loss: 0.36791015
Loss: 0.35890877
Loss: 0.35022357
Loss: 0.34185287
Loss: 0.33378
Loss: 0.32598755
Loss: 0.3184652
Loss: 0.3112234
Loss: 0.30422035
Loss: 0.29743716
Loss: 0.29086742
Loss: 0.2845134
Loss: 0.27836585
Loss: 0.2724122
Loss: 0.

In [12]:
for _ in 0..<1000 {
    let (y_pred, backprop) = classifier.appliedForBackpropagation(to: x)
    let (loss, grad_y_pred) = valueWithGradient(at: y_pred) { y_pred in softmaxCrossEntropy(logits: y_pred, labels: y) }
    print("Model output: \(y_pred), Loss: \(loss)")
    let (grad_model, _) = backprop(grad_y_pred)
    optimizer.update(&classifier, along: grad_model)
}

Streaming output truncated to the last 5000 lines.
 [   5.365037,   -2.568313, -0.83570004],
 [   5.641312,  -2.4955475,  -1.3607378],
 [   6.191916,  -3.0145254,  -0.7035156],
 [  7.6771655,  -2.9064527,   -0.908476],
 [  5.9031773,  -2.1819568,  -1.2793736],
 [   5.007953,  -1.9163282,   -1.145516],
 [  7.5238442,   -2.232231,  -2.4898362],
 [   5.882121,  -2.5225587, -0.90799415],
 [   6.095812,  -2.4398553,  -1.1061207],
 [  5.1970224,  -2.2693415,  -1.3471465],
 [    5.78195,     -2.3645,   -1.605304],
 [  5.8794975,  -1.8550785,  -2.1915197],
 [  5.3348083,  -1.7516477,  -1.4783249],
 [   7.292267,  -1.8301468,  -2.4893007],
 [  5.8427854,  -2.1036077,  -1.2386348],
 [  5.9261427,  -2.0924213,  -2.1941633],
 [   4.548314,  -1.8689747,  -1.2052209],
 [  5.5551777,  -2.1437287,  -1.5656538],
 [  4.9611406,  -2.0045676,  -1.3075979],
 [   7.627447,  -2.4942648,  -3.0535684],
 [    7.35224,  -2.2607493,  -1.9816744],
 [   9.264045,  -3.2180371,    -3.18257],
 [   5.189054,   -2.36031